In [91]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA, KernelPCA
from sklearn.linear_model import Ridge, Lasso
from sklearn.metrics import r2_score
from sklearn.kernel_ridge import KernelRidge

In [39]:
X_train = pd.read_csv("/home/richard/Documents/AML Projects/Task1/data/X_train.csv")
ids = X_train["id"]
X_train = X_train.drop(columns=["id"])

In [48]:
y_train = pd.read_csv("/home/richard/Documents/AML Projects/Task1/data/y_train.csv")['y']

In [129]:
X_train = pd.read_csv("/home/richard/Documents/AML Projects/Task1/data/X_test.csv")
ids = X_train["id"]
X_train = X_train.drop(columns=["id"])

Remove constant columns

In [130]:
X_train = X_train.loc[:, X_train.var() != 0.0]

Impute median

In [131]:
X_train = X_train.fillna(X_train.median())

Truncate

In [132]:
DELTA = 0.05
bot = X_train.quantile(DELTA)
top = X_train.quantile(1-DELTA)

In [133]:
for column in X_train.columns:
    X_train.loc[X_train[column] < bot[column], column] = bot[column]
    X_train.loc[X_train[column] > top[column], column] = top[column]

Normalize

In [134]:
X_train = (X_train - X_train.mean())/X_train.std()

Shuffle and Train Test Split

In [49]:
X_train_, X_test, y_train_, y_test = train_test_split(X_train, y_train, test_size=0.2, random_state=0)

PCA

In [112]:
pca = PCA(n_components=20)
X_train_t = pca.fit_transform(X_train_)
#print(pca.explained_variance_ratio_.sum())
#print(pca.explained_variance_ratio_[:10].sum())

In [114]:
X_test_t = pca.transform(X_test)

LASSO

In [102]:
clf = Lasso(alpha=0.5)
clf.fit(X_train_, y_train_)

Lasso(alpha=0.5)

In [103]:
(clf.coef_ > 0).sum()

20

In [104]:
X_train_t = X_train_.loc[:,clf.coef_ > 0]
X_test_t = X_test.loc[:, clf.coef_ > 0]

Ridge

In [113]:
clf = Ridge(alpha=1.0)
clf.fit(X_train_t, y_train_)

Ridge()

Evaluate

In [115]:
score = r2_score(y_test, clf.predict(X_test_t))
score

0.3603728412234507

In [116]:
r2_score(y_train_, clf.predict(X_train_t))

0.44192086113925544

In [135]:
sub = clf.predict(pca.transform(X_train))

In [136]:
pd.DataFrame({"id": ids, "y": sub}).to_csv("/home/richard/Documents/AML Projects/Task1/data/sub.csv", index=False)

In [137]:
ids

0        0.0
1        1.0
2        2.0
3        3.0
4        4.0
       ...  
771    771.0
772    772.0
773    773.0
774    774.0
775    775.0
Name: id, Length: 776, dtype: float64